In [1]:
import requests
import polars as pl

https://oldschool.runescape.wiki/w/RuneScape:Real-time_Prices

In [48]:
# recent_data_url = "https://prices.runescape.wiki/api/v1/osrs/5m"
recent_data_url = "https://prices.runescape.wiki/api/v1/osrs/latest"
mapping_url = "https://prices.runescape.wiki/api/v1/osrs/mapping"

headers = {
    'User-Agent': 'potion-arbitrage-script: @frostcow on Discord',
    'From': 'gametime785@gmail.com'  # This is another valid field
}

# (dose3_id, dose4_id)
potions = [(3034, 3032), (2454, 2452), (121, 2428),
         (22464, 22461), (22452, 22449), (9741, 9739), (6472, 6470),
         (133, 2432), (24638, 24635), (24626, 24623), (23748, 23745),
         (23736, 23733), (23700, 23697), (23688, 23685), (23724, 23721),
         (23712, 23709)]

In [49]:
recent_data_response = requests.get(recent_data_url, headers=headers)

if recent_data_response.status_code == 200:
    recent_data = recent_data_response.json()
    print("Successfully pulled recent data.")
else:
    print(f"Could not pull recent data: {recent_data_response.status_code}")

Successfully pulled recent data.


In [50]:
mapping_response = requests.get(mapping_url, headers=headers)

if mapping_response.status_code == 200:
    mapping = mapping_response.json()
    print("Successfully pulled mapping data.")
else:
    print(f"Could not pull mapping data: {mapping_response.status_code}")

Successfully pulled mapping data.


In [51]:
pl.DataFrame(mapping).select(["id", "name"])

id,name
i64,str
10344,"""3rd age amulet"""
20011,"""3rd age axe"""
12424,"""3rd age bow"""
12437,"""3rd age cloak"""
23345,"""3rd age druidic cloak"""
…,…
30321,"""Zombie helmet"""
29449,"""Zombie pirate key"""
12938,"""Zul-andra teleport"""


In [52]:
pl.DataFrame(recent_data["data"]
            ).transpose(include_header=True
            ).unnest("column_0"
            ).rename({"column": "id"}
            ).with_columns(pl.col("id").str.to_integer(base=16, strict=True)).join(
    pl.DataFrame(mapping).select(["id", "name"]), on="id", how="right")

high,highTime,low,lowTime,id,name
i64,i64,i64,i64,i64,str
null,null,null,null,10344,"""3rd age amulet"""
null,null,null,null,20011,"""3rd age axe"""
null,null,null,null,12424,"""3rd age bow"""
112,1758146842,25,1758087472,12437,"""3rd age cloak"""
null,null,null,null,23345,"""3rd age druidic cloak"""
…,…,…,…,…,…
null,null,null,null,30321,"""Zombie helmet"""
null,null,null,null,29449,"""Zombie pirate key"""
null,null,null,null,12938,"""Zul-andra teleport"""
